In [28]:
from matplotlib import pyplot as plt 
import numpy as np  
%matplotlib inline
from tqdm import tqdm
from scipy.optimize import curve_fit

In [3]:
import sys
sys.path.append('../lib')
from simlib import *

In [129]:
def mean(t, x0=0, D=1, k=1, beta=1):
    return x0 * np.exp(-D*k*beta*t)

def var(t, D=1, k=1, beta=1):
    return 1/np.sqrt(k*beta) * (1-np.exp(-2*D*k*beta*t))

In [245]:
method = 'lang'
num_particles = 1000000
max_t = 10
dt = 0.1
x0 = 2
D = 1.0
beta = 1
k = 0.5
num_steps = int(max_t/dt)

In [246]:
ts, xs = simulate(harmonic_potential(k=k), method=method, num_particles=num_particles, dt=dt, max_t=max_t, x0=x0, D=D, beta=beta)

  8%|▊         | 8/99 [00:22<04:19,  2.85s/it]

KeyboardInterrupt: 

In [241]:
m = np.mean(xs, axis=1)
v = np.var(xs, axis=1)
dm = np.std(xs, axis=1) / np.sqrt(num_particles)
dv = 2/(num_particles-1)*v**2

In [242]:
with open('../data/harmonic_validation_{}.data'.format(method), 'w') as f:
    f.write('# num particles: {}, D={}, beta={}, k={}\n'.format(num_particles, D, beta, k))
    for i, t in enumerate(ts):
        f.write('{} {} {} {} {}\n'.format(t, m[i], dm[i], v[i], dv[i]))